In [1]:
import pandas as pd
import joblib
import time

from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV  # Bayesian optimization: utilizado para optimizar hiperparámetros

import lightgbm as lgbm
from lightgbm import early_stopping  # Early stopping: utilizado para evitar sobreajuste

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\Finais\enem_censo_2023_full.pkl')

# Remover linhas com valores não explicativos
df_enem = df_enem[
    (df_enem['CAT_NACIONALIDADE'] != 0) &
    (df_enem['CAT_ENSINO'] != 0) &
    (df_enem['CAT_COR_RACA'] != 0) &
    (df_enem['CAT_ESTADO_CIVIL'] != 0) &
    (df_enem['CAT_ESCOLA'] != 1)
]

#Variaveis alvo
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajuste de tipo para MLflow -> Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

# Obter colunas categóricas
categorical_features = X_train.select_dtypes(include=['category']).columns.tolist()

# Criar Eval Set para validação cruzada (15% do conjunto de treino)
# Apenas utilizado nos modelos fianais, BayesSearchCV não utiliza Eval Set já possui validacao cruzada interna
X_train_final, X_eval, y_train_final, y_eval = train_test_split(
    X_train,
    y_train,
    test_size=0.15,
    random_state=42
)

Modelo base

In [3]:
# Treinar modelo LGBMRegressor Base
modelo_lgbm = lgbm.LGBMRegressor(n_estimators=1000, 
                                 learning_rate=0.01, 
                                 random_state=42,
                                 max_bin=4095,
                                 force_row_wise=True)

start_time = time.time()

modelo_lgbm.fit(X_train_final, 
                y_train_final['NUM_NOTA_CH'], 
                eval_set=[(X_test, y_test['NUM_NOTA_CH'])], 
                eval_metric=['r2', 'rmse', 'mae'],
                categorical_feature=categorical_features)

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 30554
[LightGBM] [Info] Number of data points in the train set: 457203, number of used features: 106
[LightGBM] [Info] Start training from score 529.413394


In [4]:
# Previsões
y_pred = modelo_lgbm.predict(X_test)

In [30]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                 parametros={**modelo_lgbm.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                 X_train=X_train,
                 y_train=y_train,
                 y_test=y_test,
                 y_pred=y_pred,
                 variavel_alvo='NUM_NOTA_CH',
                 modelo=modelo_lgbm,
                 nome_modelo='modelo_lgbm_base_censo_enem',
                 descricao_modelo='Modelo LGBMRegressor base Censo e ENEM 2023')

2025/06/07 23:04:43 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Registered model 'modelo_lgbm_base_censo_enem' already exists. Creating a new version of this model...
2025/06/07 23:04:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm_base_censo_enem, version 3
Created version '3' of model 'modelo_lgbm_base_censo_enem'.


🏃 View run casual-chimp-902 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/a0aab72192394e7198b77c23dd25713a
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_base_censo_enem
Rastreamento do MLflow finalizado.


In [5]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred, "teste")

MAE (treino): 53.6244
RMSE (treino): 68.0627
R2 (treino): 0.3488
MAE (teste): 54.9950
RMSE (teste): 69.8897
R2 (teste): 0.3151


Bayes Search

In [10]:
modelo_lgbm_bayes = lgbm.LGBMRegressor(random_state=42,
                                       max_bin=4095, 
                                       force_row_wise=True)

In [11]:
# Definição do espaço de busca para otimização bayesiana
param_grid = {
    'num_leaves': (15, 75),                         # Número de folhas na árvore de decisão
    'max_depth': (40, 120),                        # Profundidade máxima da árvore
    'learning_rate': (0.005, 0.05, 'log-uniform'), # Taxa de aprendizado
    'n_estimators': (5200, 6500),                  # Número de árvores
    'subsample': (0.1, 0.9),                       # Proporção de amostras usadas em cada árvore
    'colsample_bytree': (0.1, 0.5),                # Fração de colunas a serem usadas por árvore
    'reg_alpha': (1e-4, 0.5, 'log-uniform'),       # Regularização L1
    'reg_lambda': (5e-7, 5e-4, 'log-uniform'),     # Regularização L2
}

In [12]:
# Configurar a busca Bayesiana usando BayesSearchCV

# Criando o otimizador Bayesiano
bayes_search = BayesSearchCV(
    estimator=modelo_lgbm_bayes,    # Modelo a ser otimizado
    search_spaces=param_grid,       # Espaço de busca definido acima
    scoring='r2',                   # Critério de seleção
    n_iter=5,                       # Número de avaliações do modelo
    cv=3,                           # Validação cruzada
    random_state=42,                # Semente para reprodutibilidade
    n_jobs=-1,                      # Paralelização total dos cálculos
    verbose=1                       # 0 = sem mensagens, 1 = mensagens de progresso, 2 = mensagens detalhadas
)

In [13]:
fit_params = {
    'eval_metric': ['r2'],  #, 'rmse', 'mae'],         # Métricas a serem avaliadas
    'categorical_feature': categorical_features,       # Colunas categóricas
}

In [15]:
# Amostrar 30% do conjunto de treino para acelerar a busca bayesiana
X_bayes, _, y_bayes, _ = train_test_split(X_train, y_train, test_size=0.7, random_state=42)

In [16]:
# Executar a busca Bayesiana

start_time = time.time()
# bayes_search.fit(X_train, y_train['NUM_NOTA_CH'], **fit_params)
bayes_search.fit(X_bayes, y_bayes['NUM_NOTA_CH'], **fit_params)

# Parar o cronômetro
end_time = time.time()
elapsed_time = end_time - start_time

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[LightGBM] [Info] Total Bins 29925
[LightGBM] [Info] Number of data points in the train set: 161366, number of used features: 106
[LightGBM] [Info] Start training from score 529.385178


In [17]:
# Melhores parâmetros encontrados
try:
    melhores_parametros = bayes_search.best_params_
    print(f"Melhores parâmetros: {melhores_parametros}")
    print("R2: ", bayes_search.best_score_)
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")
except:
    melhores_parametros = {'colsample_bytree': 0.4558660098409215, 'learning_rate': 0.008293207307063736, 'max_depth': 66, 'n_estimators': 6300, 'num_leaves': 15, 'reg_alpha': 0.02296127666011783, 'reg_lambda': 5.987427535052461e-07, 'subsample': 0.7012420250723972}
    print(f"Erro ao obter melhores parâmetros, usando valores calculados anteriormente:\n {melhores_parametros}")

Melhores parâmetros: OrderedDict([('colsample_bytree', 0.4249583953429453), ('learning_rate', 0.007427481277233235), ('max_depth', 88), ('n_estimators', 6244), ('num_leaves', 46), ('reg_alpha', 0.00022546821511393962), ('reg_lambda', 9.254897279094798e-05), ('subsample', 0.7981042826479537)])
R2:  0.29516189309611746
Tempo total de execução: 2582.60 segundos


In [18]:
# Treinar o modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.set_params(**melhores_parametros)

start_time = time.time()

# Treinamento do modelo com os melhores parâmetros encontrados
modelo_lgbm_bayes.fit(X_train_final, 
                      y_train_final['NUM_NOTA_CH'], 
                      eval_set=[(X_eval, y_eval['NUM_NOTA_CH'])], 
                      eval_metric=['r2', 'rmse', 'mae'],
                      categorical_feature=categorical_features,
                      callbacks=[early_stopping(stopping_rounds=200)])

tempo_treino = time.time() - start_time

[LightGBM] [Info] Total Bins 30554
[LightGBM] [Info] Number of data points in the train set: 457203, number of used features: 106
[LightGBM] [Info] Start training from score 529.413394
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1949]	valid_0's rmse: 69.8759	valid_0's l1: 55.0321	valid_0's l2: 4882.64


In [19]:
# Previsões
y_pred_bayes = modelo_lgbm_bayes.predict(X_test)

In [20]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento,
                    modelo=modelo_lgbm_bayes,
                    parametros={**modelo_lgbm_bayes.get_params(), "amostra": X_train.shape[0], "tempo": tempo_treino},
                    X_train=X_train,
                    y_train=y_train,
                    y_test=y_test,
                    y_pred=y_pred_bayes,
                    variavel_alvo='NUM_NOTA_CH',
                    nome_modelo='modelo_lgbm_bayes_censo_enem',
                    descricao_modelo='Modelo LGBMRegressor otimizado com BayesSearchCV Censo e ENEM 2023')

2025/06/12 00:03:24 WARNING mlflow.models.signature: Failed to infer schema for inputs. Setting schema to `Schema([ColSpec(type=AnyType())]` as default. Note that MLflow doesn't validate data types during inference for AnyType. To see the full traceback, set logging level to DEBUG.
Registered model 'modelo_lgbm_bayes_censo_enem' already exists. Creating a new version of this model...
2025/06/12 00:04:12 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_lgbm_bayes_censo_enem, version 6


🏃 View run nebulous-donkey-363 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/84a7c6a3cd14474db3ec1cdff9c47840
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_lgbm_bayes_censo_enem
Rastreamento do MLflow finalizado.


Created version '6' of model 'modelo_lgbm_bayes_censo_enem'.


In [21]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_lgbm_bayes.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_bayes, "teste")

MAE (treino): 53.1981
RMSE (treino): 67.5621
R2 (treino): 0.3583
MAE (teste): 54.7792
RMSE (teste): 69.6550
R2 (teste): 0.3197


testar mais melhoria com base reduzida no bayes

In [22]:
# Salvar modelo como Pickle
joblib.dump(modelo_lgbm_bayes, 'modelos/modelo_lgbm_bayes_censo_enem.pkl')

['modelos/modelo_lgbm_bayes_censo_enem.pkl']